In [2]:
import pandas as pd
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go


Libraries

In [3]:
# Replace 'file_path.xlsx' with the path to your Excel file
file_path = '/home/sheharyar/Desktop/MLOPS/Task 4/hehe.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)


loading dataset

In [4]:
df

,Sr#,Apps Reviews,Functional suitability,Reliability,Performance Efficiency,Usability,Maintainability,Security,Compatibility,Portability,Satisfaction,Effectiveness,Freedom of Risk,Efficiency,Context Coverage,Non-Existence
0,10081,when i open app option appears update app when...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0
1,10082,when i try to sign up it always says applicati...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0
2,10083,99 of the items listed are out of stock not re...,1,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,10084,supply chain is good but they have a mass prob...,1,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,0.0
4,10085,best b2b market place ever everything is super...,1,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,10796,app stopped working i have lost all points and...,1,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0.0,0.0
20045,10797,it used to deliver where i stay now they dont ...,0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,0.0
20046,10798,i have used this app several times mostly thin...,1,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0.0,0.0
20047,10799,i ordered for the first time a few days ago i ...,1,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0.0,0.0


In [5]:
# Check for null values in the DataFrame
null_values = df.isnull().sum()

# Print the details of null values
print("Null Values in each column:")
print(null_values)

# Optionally, you can also print the percentage of null values in each column
total_rows = len(df)
null_percentage = (null_values / total_rows) * 100
print("\nPercentage of Null Values in each column:")
print(null_percentage)


Null Values in each column:
Sr#                         0
Apps Reviews                2
Functional suitability      3
Reliability                 1
Performance Efficiency      3
Usability                   5
Maintainability             5
Security                    6
Compatibility               4
Portability                 3
Satisfaction                1
Effectiveness               2
Freedom of Risk             1
Efficiency                  2
Context Coverage          123
Non-Existence               2
dtype: int64

Percentage of Null Values in each column:
Sr#                       0.000000
Apps Reviews              0.009976
Functional suitability    0.014963
Reliability               0.004988
Performance Efficiency    0.014963
Usability                 0.024939
Maintainability           0.024939
Security                  0.029927
Compatibility             0.019951
Portability               0.014963
Satisfaction              0.004988
Effectiveness             0.009976
Freedom of Risk 

In [6]:
# Drop rows with null values
df = df.dropna()



Null values handling

In [7]:
import nltk
nltk.download('stopwords')

nltk.download('punkt')
nltk.download('wordnet')
# Define and initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)


# Select columns for sentiment analysis
selected_columns = ['Apps Reviews', 'Functional suitability', 'Reliability', 'Performance Efficiency', 
                    'Usability', 'Maintainability', 'Security', 'Compatibility', 'Portability', 
                    'Satisfaction', 'Effectiveness', 'Freedom of Risk', 'Efficiency', 
                    'Non-Existence']

# Drop rows with any NaN values in selected columns
df = df.dropna(subset=selected_columns)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Apply preprocessing to the selected columns
for col in selected_columns:
    df[col] = df[col].astype(str).apply(preprocess_text)
    print(col)

# Combine selected text columns into a single text column
df['Combined_Text'] = df[selected_columns].apply(lambda row: ' '.join(row), axis=1)

# Apply sentiment analysis using TextBlob
df['Sentiment_TextBlob'] = df['Combined_Text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Define function to map continuous sentiment scores to discrete labels
def map_sentiment(score):
    if score > 0.1:
        return 'positive'
    elif score < -0.1:
        return 'negative'
    else:
        return 'neutral'

# Apply the mapping function to convert continuous sentiment scores to discrete labels
df['Sentiment_Label'] = df['Sentiment_TextBlob'].apply(map_sentiment)

# Split data into training and testing sets
X = df['Combined_Text']
y = df['Sentiment_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train Multinomial Naive Bayes classifier
nb_clf = MultinomialNB()
nb_clf.fit(X_train_tfidf, y_train)
y_pred_nb = nb_clf.predict(X_test_tfidf)

# Train Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_tfidf, y_train)
y_pred_rf = rf_clf.predict(X_test_tfidf)

# Calculate metrics for Multinomial Naive Bayes classifier
precision_nb = precision_score(y_test, y_pred_nb, average='macro')
recall_nb = recall_score(y_test, y_pred_nb, average='macro')
f1_nb = f1_score(y_test, y_pred_nb, average='macro')
accuracy_nb = accuracy_score(y_test, y_pred_nb)

print("Multinomial Naive Bayes Metrics:")
print("Precision:", precision_nb)
print("Recall:", recall_nb)
print("F1-score:", f1_nb)
print("Accuracy:", accuracy_nb)

# Calculate metrics for Random Forest classifier
precision_rf = precision_score(y_test, y_pred_rf, average='macro')
recall_rf = recall_score(y_test, y_pred_rf, average='macro')
f1_rf = f1_score(y_test, y_pred_rf, average='macro')
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("\nRandom Forest Metrics:")
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1-score:", f1_rf)
print("Accuracy:", accuracy_rf)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sheharyar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sheharyar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sheharyar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Apps Reviews
Functional suitability
Reliability
Performance Efficiency
Usability
Maintainability
Security
Compatibility
Portability
Satisfaction
Effectiveness
Freedom of Risk
Efficiency
Non-Existence
Multinomial Naive Bayes Metrics:
Precision: 0.6666724099840117
Recall: 0.6008987673777959
F1-score: 0.6174736527499384
Accuracy: 0.7108040201005025

Random Forest Metrics:
Precision: 0.803436761875049
Recall: 0.7815341303009191
F1-score: 0.7912941548903514
Accuracy: 0.8309045226130654


In [10]:
type(df['Combined_Text'][1])

str

In [ ]:
import pickle

# Assuming you have a trained model stored in a variable named 'model'
# Save the model to a file
with open('model.pkl', 'wb') as f:
    pickle.dump(rf_clf, f)

Data Preprocessing And applying model and evaluating


In [31]:

# Define colors for different sentiment labels
colors = {
    'positive': 'green',
    'negative': 'red',
    'neutral': 'blue'
}

# Create a scatter plot with custom colors for each sentiment label
fig = go.Figure()

for sentiment_label, color in colors.items():
    subset_df = df[df['Sentiment_Label'] == sentiment_label]
    fig.add_trace(go.Scatter(
        x=subset_df['Sentiment_TextBlob'],
        y=subset_df['Sentiment_Label'],
        mode='markers',
        marker=dict(size=10, color=color),
        name=sentiment_label
    ))

# Add layout settings
fig.update_layout(
    title='Sentiment Analysis Results using TextBlob',
    xaxis=dict(title='Sentiment Score'),
    yaxis=dict(title='Sentiment Label'),
    legend=dict(title='Sentiment Label'),
    autosize=False,
    width=800,
    height=500,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Show the plot
fig.show()


In [32]:
# Save DataFrame as output.csv
df.to_csv('output.csv', index=False)
df

,Sr#,Apps Reviews,Functional suitability,Reliability,Performance Efficiency,Usability,Maintainability,Security,Compatibility,Portability,Satisfaction,Effectiveness,Freedom of Risk,Efficiency,Context Coverage,Non-Existence,Combined_Text,Sentiment_TextBlob,Sentiment_Label
0,10081,open app option appears update app click optio...,0,00,0,00,00,00,00,00,00,00,0,00,0.0,10,open app option appears update app click optio...,-0.100000,neutral
1,10082,try sign always say application waiting approv...,0,00,0,00,00,00,00,00,00,00,0,00,0.0,10,try sign always say application waiting approv...,-0.500000,negative
2,10083,99 item listed stock recommended retailer find...,1,10,1,00,00,00,00,00,00,00,0,00,0.0,00,99 item listed stock recommended retailer find...,0.000000,neutral
3,10084,supply chain good mass problem stock shorting ...,1,10,1,10,10,10,10,10,10,10,1,10,1.0,00,supply chain good mass problem stock shorting ...,0.700000,positive
4,10085,best b2b market place ever everything superb q...,1,10,1,10,10,10,10,10,10,10,1,10,1.0,00,best b2b market place ever everything superb q...,1.000000,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,10796,app stopped working lost point app tell covere...,1,10,1,00,00,00,00,00,00,10,0,10,0.0,00,app stopped working lost point app tell covere...,0.000000,neutral
20045,10797,used deliver stay dont first changed using cas...,0,00,1,00,00,00,00,00,00,10,0,00,0.0,00,used deliver stay dont first changed using cas...,0.250000,positive
20046,10798,used app several time mostly thing stock im wr...,1,10,1,00,00,00,00,00,00,10,0,10,0.0,00,used app several time mostly thing stock im wr...,0.166667,positive
20047,10799,ordered first time day ago tried say dont deli...,1,10,1,00,00,00,00,00,00,10,0,10,0.0,00,ordered first time day ago tried say dont deli...,0.250000,positive


saving dataframe  output.csv